# CS4248 Project
(DON'T CLICK RUN ALL! Lemmatization takes tooo long and pls don't run it again)

In [5]:
# from google.colab import drive
# drive.mount('/content/drive')
# 
# %cd "/content/drive/My Drive/CS4248 Project"
# !cd "/content/drive/My Drive/CS4248 Project"

%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
import numpy as np
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import ComplementNB
from sklearn.feature_extraction.text import TfidfVectorizer

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/james/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
csv.field_size_limit(999999)
train = pd.read_csv('raw_data/fulltrain.csv', header = None, names=['class','text'])
X_train = train['text']
y_train = train['class']

In [8]:
train.head()

,class,text
0,1,"A little less than a decade ago, hockey fans w..."
1,1,The writers of the HBO series The Sopranos too...
2,1,Despite claims from the TV news outlet to offe...
3,1,After receiving 'subpar' service and experienc...
4,1,After watching his beloved Seattle Mariners pr...


In [9]:
test = pd.read_csv("raw_data/balancedtest.csv", header = None, names=['class','text'])
X_test = test['text']
y_test = test['class']

In [10]:
test.head()

,class,text
0,1,When so many actors seem content to churn out ...
1,1,In what football insiders are calling an unex...
2,1,In a freak accident following Game 3 of the N....
3,1,North Koreas official news agency announced to...
4,1,The former Alaska Governor Sarah Palin would b...


## Baseline Models

In [11]:
def tfidf(X_train, X_test):

  tfidf_baseline = TfidfVectorizer(ngram_range=(1,2), max_features = 30)

  X_train_tfidf = tfidf_baseline.fit_transform(X_train)

  X_test_tfidf = tfidf_baseline.transform(X_test)

  return X_train_tfidf, X_test_tfidf

X_train_tfidf, X_test_tfidf = tfidf(X_train, X_test)

In [12]:
# Logistic Regression - baseline

def LR(X_train_tfidf, y_train, X_test_tfidf, y_test):

  LR_classifier = LogisticRegression(random_state = 42, max_iter=1000).fit(X_train_tfidf, y_train)

  y_pred_lr = LR_classifier.predict(X_test_tfidf)

  print(classification_report(y_test, y_pred_lr))

  return y_pred_lr

y_pred_lr = LR(X_train_tfidf, y_train, X_test_tfidf, y_test)

              precision    recall  f1-score   support

           1       0.58      0.59      0.58       750
           2       0.57      0.21      0.31       750
           3       0.44      0.80      0.57       750
           4       0.69      0.54      0.61       750

    accuracy                           0.53      3000
   macro avg       0.57      0.53      0.52      3000
weighted avg       0.57      0.53      0.52      3000



In [13]:
# NB - baseline

def NB(X_train_tfidf, y_train, X_test_tfidf, y_test):

  nb_classifier = ComplementNB().fit(X_train_tfidf, y_train)

  y_pred_nb = nb_classifier.predict(X_test_tfidf)

  print(classification_report(y_test, y_pred_nb))

  return y_pred_nb

y_pred_nb = NB(X_train_tfidf, y_train, X_test_tfidf, y_test)

              precision    recall  f1-score   support

           1       0.54      0.58      0.56       750
           2       0.52      0.12      0.20       750
           3       0.44      0.85      0.58       750
           4       0.63      0.50      0.56       750

    accuracy                           0.51      3000
   macro avg       0.53      0.51      0.47      3000
weighted avg       0.53      0.51      0.47      3000



## Preprocessing

### Lemmatization with POS Tagging
(DON'T RUN AGAIN! It takes tooooo long)

In [14]:
# WORDNET LEMMATIZER (with appropriate pos tags)
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet

lemmatizer = WordNetLemmatizer()

# Define function to lemmatize each word with its POS tag

# POS_TAGGER_FUNCTION : TYPE 1
def pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def lemmatize_with_pos(tokens):
  # tokenize the sentence and find the POS tag for each token
  pos_tagged = nltk.pos_tag(tokens)

  # print(pos_tagged)

  wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tagged))
  # print(wordnet_tagged)

  lemmatized_sentence = []
  for word, tag in wordnet_tagged:
      if tag is None:
          # if there is no available tag, append the token as is
          lemmatized_sentence.append(word)
      else:
          # else use the tag to lemmatize the token
          lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
  # lemmatized_sentence = " ".join(lemmatized_sentence)

  # print(lemmatized_sentence)
  return lemmatized_sentence

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/james/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [15]:
# Start preprocessing

from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')
from tqdm import tqdm
from spellchecker import SpellChecker

punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

spell = SpellChecker()

def preprocessing(data):
  data_clean = []
  for sentence in tqdm(data):

    # Tokenization
    tokens = word_tokenize(sentence)

    # Remove punctuation and number
    tokens = [w for w in tokens if (not w in punc) and (not w.isdigit())]

    # Spell check
    # Taking too long time for each sentence, not practical to be used
    # tokens = [spell.correction(w) for w in tokens]
    # tokens = [w for w in tokens if w is not None and len(w) > 0]

    # Lemmatization based on tagging
    tokens = lemmatize_with_pos(tokens)

    data_clean.append((' ').join(tokens))
  return data_clean

X_train_clean = preprocessing(X_train)
X_test_clean = preprocessing(X_test)

[nltk_data] Downloading package punkt to /Users/james/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/james/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
100%|██████████| 3000/3000 [00:33<00:00, 88.40it/s] 


In [16]:
import csv
import pandas as pd

def write_to_csv(X_train_clean, X_test_clean):
    with open('raw_data/X_train_clean.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        for doc in X_train_clean:
            writer.writerow([doc])

    with open('raw_data/X_test_clean.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        for doc in X_test_clean:
            writer.writerow([doc])

# Uncomment if you want to save X_train_clean and X_test_clean
# write_to_csv(X_train_clean, X_test_clean)

In [17]:
# def read_from_file():
#     a_df = pd.read_csv('raw_data/X_train_clean.csv', header=None, dtype=str)
#     a = a_df[0].values
    
#     b_df = pd.read_csv('raw_data/X_test_clean.csv', header=None, dtype=str)
#     b = b_df[0].values
#     return a, b

# # Uncomment if you want to read from file
# a, b = read_from_file()
# X_train_clean, X_test_clean = a, b

In [18]:
X_train_clean_tfidf, X_test_clean_tfidf = tfidf(X_train_clean, X_test_clean)

_ = LR(X_train_clean_tfidf, y_train, X_test_clean_tfidf, y_test)

              precision    recall  f1-score   support

           1       0.53      0.59      0.56       750
           2       0.58      0.16      0.25       750
           3       0.46      0.78      0.58       750
           4       0.68      0.61      0.64       750

    accuracy                           0.54      3000
   macro avg       0.56      0.54      0.51      3000
weighted avg       0.56      0.54      0.51      3000



In [19]:
_ = NB(X_train_clean_tfidf, y_train, X_test_clean_tfidf, y_test)

              precision    recall  f1-score   support

           1       0.50      0.59      0.54       750
           2       0.54      0.12      0.19       750
           3       0.47      0.85      0.60       750
           4       0.66      0.53      0.59       750

    accuracy                           0.52      3000
   macro avg       0.54      0.52      0.48      3000
weighted avg       0.54      0.52      0.48      3000



### Removing stopwords, duplicates, and contradicting labels

In [20]:
def lowercase(text): #trivial casefolding
    return text.lower()

#Obtain set of stopwords from nltk
stop_words = set(stopwords.words('english'))

def remove_stopwords(data):
    data_removed_stopwords = []
    for sentence in tqdm(data):
        #tokenize sentence into words
        words = word_tokenize(sentence)
        
        #remove word if found in set of stopwords
        words_wo_stopwords = [word for word in words if word.lower() not in stop_words]
        
        data_removed_stopwords.append(' '.join(words_wo_stopwords))
    return data_removed_stopwords

def remove_dupes(data):
    #use pandas to drop duplicates, only removes those with equal values in both class and text columns
    return data.drop_duplicates()

def remove_contradictions(data):
    #group by text and filter groups with more than one unique label(contradicting labels)
    contradictions = data.groupby('text').filter(lambda x: x['class'].nunique() > 1)
    
    #remove all texts with contradicting labels since we do not know which is the correct label
    return data[~data['text'].isin(contradictions['text'])]

def preprocess2(data):
    processed_data = remove_dupes(data)
    processed_data = remove_contradictions(processed_data)
    processed_data['text'] = remove_stopwords(processed_data['text'])
    
    return processed_data

train_clean2 = preprocess2(train)    
test_clean2 = preprocess2(test)    

100%|██████████| 2990/2990 [00:04<00:00, 646.64it/s]


In [21]:
X_train_clean2 = train_clean2['text']
y_train_clean2 = train_clean2['class']
X_test_clean2 = test_clean2['text']
y_test_clean2 = test_clean2['class']

X_train_clean2_tfidf, X_test_clean2_tfidf = tfidf(X_train_clean2, X_test_clean2)

In [22]:
_ = LR(X_train_clean2_tfidf, y_train_clean2, X_test_clean2_tfidf, y_test_clean2)

              precision    recall  f1-score   support

           1       0.49      0.52      0.51       745
           2       0.52      0.08      0.14       750
           3       0.38      0.76      0.51       747
           4       0.57      0.45      0.50       748

    accuracy                           0.45      2990
   macro avg       0.49      0.45      0.42      2990
weighted avg       0.49      0.45      0.42      2990



In [23]:
_ = NB(X_train_clean2_tfidf, y_train_clean2, X_test_clean2_tfidf, y_test_clean2)

              precision    recall  f1-score   support

           1       0.41      0.55      0.47       745
           2       0.41      0.12      0.18       750
           3       0.38      0.69      0.49       747
           4       0.58      0.33      0.42       748

    accuracy                           0.42      2990
   macro avg       0.45      0.42      0.39      2990
weighted avg       0.45      0.42      0.39      2990



## Feature Engineering

### Text subjectivity

In [24]:
# Text Subjectivity (w/o preprocessing)
from textblob import TextBlob

def subjectivity(data, tf_idf):
    # new_data = data.copy()
    new_data = data
    new_data['psycho-linguistic'] = new_data["text"].apply(lambda x: TextBlob(x).sentiment)
    new_data['subjectivity'] = new_data['psycho-linguistic'].apply(lambda x: x[1])
    tfidf_df = pd.DataFrame(tf_idf.toarray())
    subject_df = pd.merge(tfidf_df, new_data['subjectivity'],left_index=True, right_index=True)
    subject_df.columns = subject_df.columns.astype(str)
    return subject_df

train_subject_df = subjectivity(train, X_train_tfidf)
test_subject_df = subjectivity(test, X_test_tfidf)

In [25]:
_ = LR(train_subject_df, y_train, test_subject_df, y_test)

              precision    recall  f1-score   support

           1       0.61      0.62      0.62       750
           2       0.55      0.19      0.29       750
           3       0.44      0.80      0.57       750
           4       0.71      0.56      0.63       750

    accuracy                           0.55      3000
   macro avg       0.58      0.54      0.52      3000
weighted avg       0.58      0.55      0.52      3000



In [26]:
_ = NB(train_subject_df, y_train, test_subject_df, y_test)

              precision    recall  f1-score   support

           1       0.57      0.59      0.58       750
           2       0.52      0.12      0.19       750
           3       0.44      0.85      0.58       750
           4       0.64      0.51      0.57       750

    accuracy                           0.52      3000
   macro avg       0.54      0.52      0.48      3000
weighted avg       0.54      0.52      0.48      3000



In [27]:
# Text Subjectivity (with preprocessing)
train_clean_subject_df = subjectivity(train, X_train_clean_tfidf)
test_clean_subject_df = subjectivity(test, X_test_clean_tfidf)

In [28]:
_ = LR(train_clean_subject_df, y_train, test_clean_subject_df, y_test)

              precision    recall  f1-score   support

           1       0.56      0.65      0.60       750
           2       0.56      0.15      0.24       750
           3       0.46      0.79      0.58       750
           4       0.71      0.62      0.66       750

    accuracy                           0.55      3000
   macro avg       0.57      0.55      0.52      3000
weighted avg       0.57      0.55      0.52      3000



In [29]:
_ = NB(train_clean_subject_df, y_train, test_clean_subject_df, y_test)

              precision    recall  f1-score   support

           1       0.51      0.59      0.55       750
           2       0.54      0.12      0.20       750
           3       0.47      0.85      0.61       750
           4       0.66      0.54      0.59       750

    accuracy                           0.53      3000
   macro avg       0.55      0.53      0.49      3000
weighted avg       0.55      0.53      0.49      3000



### TF-IDF Vectorizer
We will adjust the possible parameters, i.e. n_grams, max_features here.

In [30]:
# All this need Kfold cross validation in the future

n_grams = [1, 2] # 3, 4]
max_features = 29 #[10000, 20000, None]
def tfidf(X_train, X_test, n_gram, max_feature):
  tfidf_baseline = TfidfVectorizer(ngram_range=(1,n_gram), max_features = max_feature)
  X_train_tfidf = tfidf_baseline.fit_transform(X_train)
  X_test_tfidf = tfidf_baseline.transform(X_test)
  return X_train_tfidf, X_test_tfidf

for n_gram in n_grams:
  for max_feature in max_features:
    X_train_tfidf, X_test_tfidf = tfidf(X_train_clean, X_test_clean, n_gram, max_feature)
    print(f"n_gram: {n_gram}, max_feature: {max_feature}")
    num_features = X_train_tfidf.shape[1]
    if max_feature is None:
      print(f"Number of features: {num_features}")
    _ = LR(X_train_tfidf, y_train, X_test_tfidf, y_test)
    _ = NB(X_train_tfidf, y_train, X_test_tfidf, y_test)

# Ran until n_gram = 3 for 30min ><

TypeError: 'int' object is not iterable

### Word Embeddings
#### GloVe
- Download from https://nlp.stanford.edu/projects/glove/
- Used Wikipedia 2014 + Gigaword 5

In [ ]:
wordVec = KeyedVectors.load("word2vec-google-news-300.model", mmap='r')
def meanEmbed(text, wordVec):
    mean = []
    
    for t in text:
        wordVecs = []

        for word in t.split():
            if word in wordVec:
                wordVecs.append(wordVec[word])
    
        if not wordVecs:
            wordVecs.append(np.zeros(wordVec.vector_size))

        tempmean = np.mean(wordVecs, axis=0)
        mean.append(tempmean)

    return np.array(mean)

In [ ]:
def load_glove_model(glove_file_path):
    glove_model = {}
    with open(glove_file_path, 'r', encoding="utf8") as f:
        for line in f:
            split_line = line.split()
            word = split_line[0]
            # embedding = np.array([float(val) for val in split_line[1:]])
            try:
                coefs = np.asarray(split_line[1:], dtype='float32')
            except ValueError:
                pass
            glove_model[word] = coefs
    return glove_model

# glove_model = load_glove_model("glove.6B/glove.6B.300d.txt")


FileNotFoundError: [Errno 2] No such file or directory: 'glove.6B/glove.6B.300d.txt'

In [ ]:
# print(glove_model['hello'].shape)
# print(glove_model['hello'][:5])

(300,)
[-0.33712   -0.21691   -0.0066365 -0.41625   -1.2555   ]


In [ ]:
# def text_to_features(tokens, glove_model):
#     vectors = [glove_model[token] for token in tokens if token in glove_model]
#     if vectors:
#         return np.mean(vectors, axis=0) # Return the mean of the vectors
#     else:
#         return np.zeros(300)  # Return a zero vector if no words are found

# # Convert X_train_clean to a pandas Series
# X_train_clean_series = pd.Series(X_train_clean)
# # Convert text to features using a predefined function (e.g., using GloVe embeddings)
# glove_embedding_features = X_train_clean_series.apply(lambda x: pd.Series(text_to_features(x, glove_model)))
# glove_embedding_features.columns = [f'Feature_{i}' for i in range(300)]

# # Use the same function to convert X_test_clean to a pandas Series
# X_test_clean_series = pd.Series(X_test_clean)
# glove_embedding_features_test = X_test_clean_series.apply(lambda x: pd.Series(text_to_features(x, glove_model)))
# glove_embedding_features_test.columns = [f'Feature_{i}' for i in range(300)]

# # Runs for a short while : ~45s on my machine

In [ ]:
# print(glove_embedding_features.head())
# print(glove_embedding_features.shape)

   Feature_0  Feature_1  Feature_2  Feature_3  Feature_4  Feature_5  \
0  -0.246974   0.048534  -0.288494  -0.078190  -0.596826  -0.001970   
1  -0.237516   0.038033  -0.291854  -0.056598  -0.580772  -0.006075   
2  -0.256160   0.036380  -0.281820  -0.076163  -0.587690   0.000572   
3  -0.244585   0.044431  -0.285958  -0.077551  -0.579946  -0.011869   
4  -0.262932   0.050881  -0.302546  -0.065594  -0.586354  -0.021741   

   Feature_6  Feature_7  Feature_8  Feature_9  ...  Feature_290  Feature_291  \
0  -0.074773   0.236836  -0.342276  -1.160948  ...    -0.159519     0.036412   
1  -0.077081   0.216868  -0.343079  -1.178258  ...    -0.152523     0.026440   
2  -0.061376   0.217344  -0.324343  -1.165821  ...    -0.151057     0.010553   
3  -0.065669   0.223504  -0.321919  -1.182414  ...    -0.152639     0.010548   
4  -0.074013   0.224716  -0.313182  -1.172550  ...    -0.161936     0.016166   

   Feature_292  Feature_293  Feature_294  Feature_295  Feature_296  \
0    -0.198667    -0.0

In [ ]:
# # Cleaned data + GloVe embeddings + Logistic Regression
# _ = LR(glove_embedding_features, y_train, glove_embedding_features_test, y_test)

# # Cleaned data + GloVe embeddings + Naive Bayes
# # _ = NB(glove_embedding_features, y_train, glove_embedding_features_test, y_test)
# # ValueError: Negative values in data passed to ComplementNB (input X)

# # Cleaned data + GloVe embeddings + Naive Bayes (as a workaround)
# from sklearn.naive_bayes import GaussianNB
# classifier = GaussianNB()
# _ = classifier.fit(glove_embedding_features.to_numpy(), y_train)
# y_pred = classifier.predict(glove_embedding_features_test.to_numpy())
# print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.22      0.23      0.22       750
           2       0.57      0.22      0.32       750
           3       0.19      0.33      0.24       750
           4       0.66      0.53      0.59       750

    accuracy                           0.33      3000
   macro avg       0.41      0.33      0.34      3000
weighted avg       0.41      0.33      0.34      3000

              precision    recall  f1-score   support

           1       0.24      0.28      0.26       750
           2       0.37      0.85      0.52       750
           3       0.03      0.01      0.01       750
           4       0.65      0.25      0.36       750

    accuracy                           0.35      3000
   macro avg       0.32      0.35      0.29      3000
weighted avg       0.32      0.35      0.29      3000


### Hedging word count

In [ ]:
# Hedging Word Count
# Reference: all 176 hedging words in hedges.txt are from https://github.com/words/hedges
import re

def fetch_hedging_regex(filename='hedges.txt'):
    hedges = []
    file = open(filename, 'r')
    lines = file.readlines()
    for l in lines:
        hedges.append(l.strip())
    hedges_re = "(?:^|\W)(" + "|".join(hedges) + ")(?:$|\W)"
    return hedges_re

def hedges(x, hedges_re):
    res = re.findall(hedges_re, x.lower())
    counts = sum(1 for s in res if s != '')
    if counts > 0:
        return 1 
    return 0

def hedging_word_count(data, tf_idf, hedge_re):
    new_data = data.copy()
    new_data["hedges"] = new_data.loc[:, "text"].apply(hedges, hedges_re=hedge_re)
    tfidf_df = pd.DataFrame(tf_idf.toarray())
    hedged_df = pd.merge(tfidf_df, new_data['hedges'],left_index=True, right_index=True)
    hedged_df.columns = hedged_df.columns.astype(str)
    return hedged_df

hedges_re = fetch_hedging_regex()
train_hedge_df = hedging_word_count(train, X_train_tfidf, hedges_re)
test_hedge_df = hedging_word_count(test, X_test_tfidf, hedges_re)

NameError: name 'train' is not defined

In [ ]:
_ = LR(train_hedge_df, y_train, test_hedge_df, y_test)

NameError: name 'LR' is not defined

In [ ]:
_ = NB(train_hedge_df, y_train, test_hedge_df, y_test)

              precision    recall  f1-score   support

           1       0.62      0.62      0.62       750
           2       0.62      0.46      0.53       750
           3       0.62      0.93      0.74       750
           4       0.88      0.67      0.76       750

    accuracy                           0.67      3000
   macro avg       0.69      0.67      0.66      3000
weighted avg       0.69      0.67      0.66      3000


In [ ]:
# Hedging Word Count (with preprocessing)
train_clean_hedge_df = hedging_word_count(train, X_train_clean_tfidf, hedges_re)
test_clean_hedge_df = hedging_word_count(test, X_test_clean_tfidf, hedges_re)

NameError: name 'train' is not defined

In [ ]:
_ = LR(train_clean_hedge_df, y_train, test_clean_hedge_df, y_test)

              precision    recall  f1-score   support

           1       0.85      0.83      0.84       750
           2       0.83      0.41      0.54       750
           3       0.59      0.82      0.69       750
           4       0.81      0.91      0.86       750

    accuracy                           0.74      3000
   macro avg       0.77      0.74      0.73      3000
weighted avg       0.77      0.74      0.73      3000


In [ ]:
_ = NB(train_clean_hedge_df, y_train, test_clean_hedge_df, y_test)

              precision    recall  f1-score   support

           1       0.60      0.61      0.61       750
           2       0.64      0.51      0.57       750
           3       0.63      0.91      0.75       750
           4       0.87      0.64      0.74       750

    accuracy                           0.67      3000
   macro avg       0.69      0.67      0.67      3000
weighted avg       0.69      0.67      0.67      3000


### Text Sentiment

In [ ]:
nltk.download('vader_lexicon')

NameError: name 'nltk' is not defined

In [ ]:
import pandas as pd
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer

sia = SentimentIntensityAnalyzer()

def sentiment_features(data, tf_idf):
    new_data = data.copy()
    
    new_data['sentiments'] = new_data['text'].apply(lambda x: sia.polarity_scores(x))
    new_data['compound'] = new_data['sentiments'].apply(lambda x: x['compound'])
    tfidf_df = pd.DataFrame(tf_idf.toarray())
    sentiment_df = pd.merge(tfidf_df, new_data['compound'], left_index=True, right_index=True)
    sentiment_df.columns = sentiment_df.columns.astype(str)
    
    return sentiment_df

train_sentiment_df = sentiment_features(train, X_train_tfidf)
test_sentiment_df = sentiment_features(test, X_test_tfidf)

In [ ]:
_ = LR(train_sentiment_df, y_train, test_sentiment_df)

              precision    recall  f1-score   support

           1       0.85      0.80      0.82       750
           2       0.82      0.37      0.52       750
           3       0.56      0.83      0.67       750
           4       0.81      0.92      0.86       750

    accuracy                           0.73      3000
   macro avg       0.76      0.73      0.72      3000
weighted avg       0.76      0.73      0.72      3000


In [ ]:
# Sentiment Scores may be negative so NB will not work. For NB, I have added a constant to the sentiment scores to make them all positive.
min_compound_train = train_sentiment_df['compound'].min()
min_compound_test = test_sentiment_df['compound'].min()
min_compound = min(min_compound_train, min_compound_test)

if min_compound < 0:
    adjustment = abs(min_compound)
    train_sentiment_df['compound'] += adjustment
    test_sentiment_df['compound'] += adjustment

_ = NB(train_sentiment_df, y_train, test_sentiment_df)

              precision    recall  f1-score   support

           1       0.61      0.64      0.62       750
           2       0.62      0.45      0.52       750
           3       0.62      0.92      0.74       750
           4       0.88      0.66      0.75       750

    accuracy                           0.66      3000
   macro avg       0.68      0.66      0.66      3000
weighted avg       0.68      0.66      0.66      3000


In [ ]:
#With Preprocessing
train_clean_sentiment_df = sentiment_features(train, X_train_clean_tfidf)
test_clean_sentiment_df = sentiment_features(test, X_test_clean_tfidf)

In [ ]:
_ = LR(train_clean_sentiment_df, y_train, test_clean_sentiment_df y_test)

              precision    recall  f1-score   support

           1       0.84      0.81      0.82       750
           2       0.84      0.42      0.56       750
           3       0.59      0.83      0.69       750
           4       0.81      0.91      0.86       750

    accuracy                           0.74      3000
   macro avg       0.77      0.74      0.73      3000
weighted avg       0.77      0.74      0.73      3000


In [ ]:
# Sentiment Scores may be negative so NB will not work. For NB, I have added a constant to the sentiment scores to make them all positive.
min_clean_compound_train = train_clean_sentiment_df['compound'].min()
min_clean_compound_test = test_clean_sentiment_df['compound'].min()
min_clean_compound = min(min_clean_compound_train, min_clean_compound_test)

if min_clean_compound < 0:
    adjustment = abs(min_clean_compound)
    train_clean_sentiment_df['compound'] += adjustment
    test_clean_sentiment_df['compound'] += adjustment
    
_ = NB(train_clean_sentiment_df, y_train, test_clean_sentiment_df, y_test)

              precision    recall  f1-score   support

           1       0.60      0.62      0.61       750
           2       0.63      0.48      0.54       750
           3       0.62      0.91      0.74       750
           4       0.87      0.64      0.74       750

    accuracy                           0.66      3000
   macro avg       0.68      0.66      0.66      3000
weighted avg       0.68      0.66      0.66      3000


### Additional Features

- Sentence Length
- Number of quotations
- Count of word types:
    - Numbers
    - Proper nouns
    - Conjunctions
    - Superlatives
    - 1st / 3rd pronoun counts


In [ ]:
# Again, all this need Kfold cross validation in the future

# Count sentence length
# Count number of quotations
# Count of word types: Numbers, Proper nouns, Conjunctions, Superlatives, 1st / 3rd pronoun counts (w/o preprocessing)
# Count of booster words (with preprocessing)

import pandas as pd
import numpy as np
import re
import nltk
from scipy.sparse import hstack
import time

# Uncomment if you have not downloaded the 'punkt' and 'averaged_perceptron_tagger' packages
# nltk.download('punkt') 
# nltk.download('averaged_perceptron_tagger')

def add_custom_feature_to_tfidf(data, tfidf_matrix, feature_extraction_func):
    custom_feature = data.apply(feature_extraction_func)
    custom_feature_sparse = np.array(custom_feature).reshape(-1, 1)
    enhanced_tfidf_matrix = hstack([tfidf_matrix, custom_feature_sparse])
    return enhanced_tfidf_matrix

# Feature extraction functions
def count_sentence_length(text):
    return len(nltk.sent_tokenize(text))

def count_quotations(text):
    return text.count('"')

def count_numbers(text):
    return len(re.findall(r'\d+', text))

def count_proper_nouns(text):
    tagged = nltk.pos_tag(nltk.word_tokenize(text))
    proper_nouns = [word for word, pos in tagged if pos == 'NNP']
    return len(proper_nouns)

def count_conjunctions(text):
    tagged = nltk.pos_tag(nltk.word_tokenize(text))
    conjunctions = [word for word, pos in tagged if pos == 'CC']
    return len(conjunctions)

def count_superlatives(text):
    tagged = nltk.pos_tag(nltk.word_tokenize(text))
    superlatives = [word for word, pos in tagged if pos == 'JJS']
    return len(superlatives)

def count_1st_pronouns(text):
    tagged = nltk.pos_tag(nltk.word_tokenize(text))
    first_pronouns = [word for word, pos in tagged if pos == 'PRP' and word.lower() in ['i', 'me', 'my', 'mine']]
    return len(first_pronouns)

def count_3rd_pronouns(text):
    tagged = nltk.pos_tag(nltk.word_tokenize(text))
    third_pronouns = [word for word, pos in tagged if pos == 'PRP' and word.lower() in 
                      ['he', 'him', 'his', 'she', 'her', 'hers', 'it', 'its', 'they', 'them', 'their', 'theirs']]
    return len(third_pronouns)

def count_booster_words(text):
    booster_words = """absolutely amazingly awfully completely considerably definitely entirely exceedingly
                    extremely highly incredibly indeed quite really remarkably so terribly totally truly utterly very"""
    booster_words = booster_words.split(" ")
    return len([word for word in text.split(" ") if word in booster_words])

# Evaluate the performance of the model with the custom features
# For the purpose of comparison, we can just use the logistic regression model across all the custom features
features_methods_dictionary =  {
    "Sentence Length": count_sentence_length,
    "Count Quotations": count_quotations,
    "Count Numbers": count_numbers,
    "Count Proper Nouns": count_proper_nouns,
    "Count Conjunctions": count_conjunctions,
    "Count Superlatives": count_superlatives,
    "Count 1st Pronouns": count_1st_pronouns,
    "Count 3rd Pronouns": count_3rd_pronouns,
    "Count Booster Words": count_booster_words
}

training_datasets = {
    "Not Processed": [X_train, X_train_tfidf, X_test, X_test_tfidf],
    "Processed": [X_train_clean, X_train_clean_tfidf, X_test_clean, X_test_clean_tfidf]
}
for dataset_name , datasets in training_datasets.items():
    X_train = datasets[0]
    X_train_tfidf = datasets[1]
    X_test = datasets[2]
    X_test_tfidf = datasets[3]
    for feature_name, feature_matrix in features_methods_dictionary.items():
        time_start = time.time()
        if dataset_name == "Not Processed":
            X_train_enhanced = add_custom_feature_to_tfidf(X_train, X_train_tfidf, feature_matrix)
            X_test_enhanced = add_custom_feature_to_tfidf(X_test, X_test_tfidf, feature_matrix)
        else:
            X_train_enhanced = add_custom_feature_to_tfidf(X_train, X_train_tfidf, feature_matrix)
            X_test_enhanced = add_custom_feature_to_tfidf(X_test, X_test_tfidf, feature_matrix)
        _ = LR(X_train_enhanced, y_train, X_test_enhanced, y_test)
        time_end = time.time()
        print(f"Time taken for {feature_name} on {dataset_name} dataset: {time_end - time_start} seconds")


/Users/zhizhouhuang/Desktop/Labeled-Unreliable-News/venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

           1       0.86      0.84      0.85       750
           2       0.84      0.44      0.58       750
           3       0.60      0.82      0.69       750
           4       0.81      0.91      0.86       750

    accuracy                           0.75      3000
   macro avg       0.78      0.75      0.74      3000
weighted avg       0.78      0.75      0.74      3000

Time taken for Sentence Length on Not Processed dataset: 93.36047506332397 seconds
              precision    recall  f1-score   support

           1       0.85      0.79      0.82       750
           2       0.81      0.37      0.51       750
           3       0.56      0.83      0.67       750
           4       0.81      0.92      0.86       750

    accuracy                           0.73      3000
   macro avg       0.76      0.73      0.71      3000
weighted avg       0.76      0.73      0.71      3000

Time taken for Count Quotations on Not Processe

/Users/zhizhouhuang/Desktop/Labeled-Unreliable-News/venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

           1       0.85      0.81      0.83       750
           2       0.84      0.43      0.56       750
           3       0.58      0.80      0.68       750
           4       0.79      0.91      0.85       750

    accuracy                           0.74      3000
   macro avg       0.76      0.74      0.73      3000
weighted avg       0.76      0.74      0.73      3000

Time taken for Count Proper Nouns on Not Processed dataset: 672.8190791606903 seconds
              precision    recall  f1-score   support

           1       0.85      0.81      0.83       750
           2       0.82      0.42      0.56       750
           3       0.59      0.83      0.69       750
           4       0.81      0.91      0.86       750

    accuracy                           0.74      3000
   macro avg       0.77      0.74      0.74      3000
weighted avg       0.77      0.74      0.74      3000

Time taken for Count Conjunctions on Not Pro

KeyboardInterrupt: 

In [ ]:
from sklearn.preprocessing import StandardScaler

def scale_features(train_df, test_df):
    scaler = StandardScaler()
    train_scaled = scaler.fit_transform(train_df)
    test_scaled = scaler.transform(test_df)
    
    train_scaled_df = pd.DataFrame(train_scaled, columns=train_df.columns)
    test_scaled_df = pd.DataFrame(test_scaled, columns=test_df.columns)
    
    return train_scaled_df, test_scaled_df

train_length_scaled_df, test_length_scaled_df = scale_features(X_train_enhanced, X_test_enhanced)



In [ ]:
_ = LR(train_length_scaled_df, y_train, test_length_scaled_df, y_test)

              precision    recall  f1-score   support

           1       0.84      0.74      0.79       750
           2       0.73      0.40      0.51       750
           3       0.57      0.73      0.64       750
           4       0.74      0.94      0.82       750

    accuracy                           0.70      3000
   macro avg       0.72      0.70      0.69      3000
weighted avg       0.72      0.70      0.69      3000


In [ ]:
# For some reason, after scaling sentence length can give negative values so I will just use NB with unscaled sentence length.

_ = NB(X_train_enhanced, y_train, X_test_enhanced, y_test)

              precision    recall  f1-score   support

           1       0.65      0.86      0.74       750
           2       0.72      0.42      0.53       750
           3       0.56      0.93      0.70       750
           4       0.90      0.39      0.55       750

    accuracy                           0.65      3000
   macro avg       0.71      0.65      0.63      3000
weighted avg       0.71      0.65      0.63      3000
